# arXiv
## input:
##### txt_semRefTitulo:
pasta onde cada arquivo .txt contem o conteúdo de um artigo do arXiv desconsiderando resumo e título e o nome do arquivo é o próprio dódigo identificador do arquivo.
##### titulo_doc_comunidade.csv:
Associa identificador do artido ao seu título e comunidade
##### titlesAutores___.csv, 
Associa identificador do autor do grafo que identificou as comunidades e os títulos dos artigos associados a ele
## objetivo:
gerar um arquivo .txt para cada autor com o conteúdo de todos os artigos relacinados a ele e a partir desses arquivos gerar a matrix de dados.

Caminho:

(1) coletar títulos dos artigos associados a cada um deles (arquivo itlesAutores____.csv):

(2) a partir do título identificar o código do artigo (titlesAutores____.csv e titulo_doc_comunidade.csv):

(3) a partir do código do artigo, baixar o conteúdo no arquivo .txt (pasta txt_semRefTitulo)

(4) salvar o conteúdo de todos os artigos num novo arquivo .txt que terá como nome o código do autor (pasta) txt_autores_id

(5) rodar pipeline de Renato a partir de (4)

## OBS:
script deve ser rodado para cada tipo de base titlesAutores___: titlesAutoresGlobal, titlesAutoresGrau, titlesAutoresPagerank, resultando em bases diferentes. Esvaziar pasta txt_autores_id antes de rodar de novo.

In [349]:
import pandas as pd
import requests as rq
import numpy as np
from bs4 import BeautifulSoup
from time import sleep
import re
from time import sleep
import os.path
import re

In [376]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII').replace(' ', '')
def letras_apenas(txt):
    texto = normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
    texto = re.sub('[^a-zA-Z]', ' ', texto) #mantendo letras e números: ex. Jose S. -> JoseS
    return texto

In [377]:
titulos_com_autores = pd.read_csv('titulo_doc_comunidade.csv',sep=';')
titulos_com_autores.head()

,title,community,id
0,Plausible reasoning from spatial observations,6,1301.2285v1
1,A Monte-Carlo Algorithm for Dempster-Shafer Be...,6,1303.5757v1
2,The Computational Complexity of Dominance and ...,6,1401.3453v1
3,The Assumptions Behind Dempster's Rule,6,1303.1518v1
4,Generating Graphoids from Generalised Conditio...,6,1302.6852v1


In [378]:
titulos_com_autores.shape

(571, 3)

In [379]:
com_idAutores_totulo = pd.read_csv('titlesAutoresGlobal.csv',sep='@')
com_idAutores_totulo.head()

,Community,uuthor_id,Titles
0,6,58553,Nonparametric Bayesian Logic;Constraint Proces...
1,6,6474,Learning Belief Networks in Domains with Recur...
2,6,58554,Inter-causal Independence and Heterogeneous Fa...
3,6,31206,A Monte-Carlo Algorithm for Dempster-Shafer Be...
4,6,78646,Possibilistic Constraint Satisfaction Problems...


In [380]:
com_idAutores_totulo['Titles'] = com_idAutores_totulo['Titles'].str.split(';')

In [381]:
com_idAutores_totulo.head()

,Community,uuthor_id,Titles
0,6,58553,"[Nonparametric Bayesian Logic, Constraint Proc..."
1,6,6474,[Learning Belief Networks in Domains with Recu...
2,6,58554,[Inter-causal Independence and Heterogeneous F...
3,6,31206,[A Monte-Carlo Algorithm for Dempster-Shafer B...
4,6,78646,[Possibilistic Constraint Satisfaction Problem...


### Criando dicionário que mapeia título do artigo com identificador

In [382]:
dicionario = {'titulo':'id'}
dicionario2 = {'titulo':'id'}
for index, row in com_idAutores_totulo.iterrows():
    com, id_autor, titulos = row
    for t in titulos:
        t = remover_acentos(t)
        dicionario2[t] = '' #teste
        for i, r in titulos_com_autores.iterrows():
            titulo2, com2, ids_titulo2 = r
            titulo2 = remover_acentos(titulo2)
            if t == titulo2:
                dicionario[t] = ids_titulo2

In [383]:
len(dicionario)

269

In [384]:
len(dicionario2)

269

In [307]:
#dicionario

Diferença entre len(dicionario) e len(dicionario2) mostra que alguns títulos vão se perder, mas que eles serão poucos.

### Gerando arquivos

In [385]:
from time import sleep
for index, row in com_idAutores_totulo.iterrows():
    com, id_autor, titulos = row
    
    nome = 'txt_autores_id\\' + str(id_autor) + '_' + str(com) + '.txt'
    print('salvando em ' + nome)
    fileAutor = open(nome,"w+",encoding='utf-8')
    for t in titulos:
        t = remover_acentos(t)
        idArtigo = dicionario[t]
        print(idArtigo + ':' + t)
        #capturando conteúdo do artigo
        artigo = 'txt_semRefTitulo\\' + str(idArtigo) + '.txt' 
        file = open(artigo,"r",encoding='utf-8')
        texto = file.read()
        texto = letras_apenas(texto)
        fileAutor.write(texto)
        fileAutor.write('\n')
        file.close()
        sleep(0.05)
    fileAutor.close()        

salvando em txt_autores_id\58553_6.txt
1207.1375v1:NonparametricBayesianLogic
1205.2635v1:ConstraintProcessinginLiftedProbabilisticInference
salvando em txt_autores_id\6474_6.txt
1302.1549v1:LearningBeliefNetworksinDomainswithRecursivelyEmbeddedPseudoIndependentSubmodels
1201.2250v1:ARobustQuantumRandomAccessMemory
1302.1529v1:ExploringParallelisminLearningBeliefNetworks
1303.5438v1:ExploringLocalizationinBayesianNetworksforLargeExpertSystems
salvando em txt_autores_id\58554_6.txt
1302.6855v1:Inter-causalIndependenceandHeterogeneousFactorization
1304.2370v1:ProbabilisticSemanticsandDefaults
1301.2309v1:SymmetricCollaborativeFilteringUsingtheNoisySensorModel
1207.1375v1:NonparametricBayesianLogic
1205.2635v1:ConstraintProcessinginLiftedProbabilisticInference
1205.2651v1:SeeingtheForestDespitetheTrees:LargeScaleSpatial-TemporalDecisionMaking
salvando em txt_autores_id\31206_6.txt
1303.5757v1:AMonte-CarloAlgorithmforDempster-ShaferBelief
1401.3453v1:TheComputationalComplexityofDominancean

salvando em txt_autores_id\77032_104.txt
1206.5928v1:CAPIR:CollaborativeActionPlanningwithIntentionRecognition
1303.5730v1:RepresentationRequirementsforSupportingDecisionModelFormulation
1303.5414v1:RepresentingContext-SensitiveKnowledgeinaNetworkFormalism:APreliminaryReport
1301.3872v1:CausalMechanism-basedModelConstruction
1206.5940v1:BootstrappingMonteCarloTreeSearchwithanImperfectHeuristic
salvando em txt_autores_id\100199_104.txt
1303.1480v1:UsingFirst-OrderProbabilityLogicfortheConstructionofBayesianNetworks
1302.6826v1:UsingNewDatatoRefineaBayesianNetwork
1303.1483v1:UsingCausalInformationandLocalMeasurestoLearnBayesianNetworks
1302.6783v1:GeneratingNewBeliefsFromOld
1301.2259v1:UCP-Networks:ADirectedGraphicalRepresentationofConditionalUtilities
1304.2341v1:ProbabilityDistributionsOverPossibleWorlds
1401.3458v1:Solving#SATandBayesianInferencewithBacktrackingSearch
salvando em txt_autores_id\31138_116.txt
1202.3718v1:OntheComplexityofDecisionMakinginPossibilisticDecisionTrees
120

1301.2313v1:BayesianError-BarsforBeliefNetInference
1206.4654v1:AGeneralizedLoopCorrectionMethodforApproximateInferenceinGraphicalModels
1405.1436v1:TrainingRestrictedBoltzmannMachinebyPerturbation
salvando em txt_autores_id\58566_145.txt
1205.2642v1:ImprovedMeanandVarianceApproximationsforBeliefNetResponsesviaNetworkDoubling
salvando em txt_autores_id\63480_145.txt
1301.2343v1:PlanningbyPrioritizedSweepingwithSmallBackups
1206.3285v1:Dyna-StylePlanningwithLinearFunctionApproximationandPrioritizedSweeping
1205.4839v5:Off-PolicyActor-Critic
salvando em txt_autores_id\72512_145.txt
1206.3233v1:SpeedingUpPlanninginMarkovDecisionProcessesviaAutomaticallyConstructedAbstractions
salvando em txt_autores_id\72513_145.txt
1206.3233v1:SpeedingUpPlanninginMarkovDecisionProcessesviaAutomaticallyConstructedAbstractions
salvando em txt_autores_id\72582_145.txt
1206.3285v1:Dyna-StylePlanningwithLinearFunctionApproximationandPrioritizedSweeping
salvando em txt_autores_id\72583_145.txt
1207.1411v1:Baye

1302.6797v1:OntheRelationbetweenKappaCalculusandProbabilisticReasoning
1302.6796v1:ActionNetworks:AFrameworkforReasoningaboutActionsandChangeunderUncertainty
1303.5406v1:ReasoningWithQualitativeProbabilitiesCanBeTractable
1206.5279v1:Makinglifebetteronelargesystematatime:ChallengesforUAIresearch
salvando em txt_autores_id\72574_156.txt
1206.3280v1:CT-NOR:RepresentingandReasoningAboutEventsinContinuousTime
salvando em txt_autores_id\72575_156.txt
1206.3280v1:CT-NOR:RepresentingandReasoningAboutEventsinContinuousTime
salvando em txt_autores_id\72576_156.txt
1206.3280v1:CT-NOR:RepresentingandReasoningAboutEventsinContinuousTime
salvando em txt_autores_id\72577_156.txt
1206.3280v1:CT-NOR:RepresentingandReasoningAboutEventsinContinuousTime
salvando em txt_autores_id\72578_156.txt
1206.3280v1:CT-NOR:RepresentingandReasoningAboutEventsinContinuousTime
salvando em txt_autores_id\72561_256.txt
1206.6818v1:SensitivityAnalysisforThresholdDecisionMakingwithDynamicNetworks
1301.3889v1:PivotalPrunin

# Pipeline Renato a partir dos arquivos .txt

In [386]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

In [387]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(re.sub('[-\xad][\s\n]','',text)) \
                          for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]{3,}', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
    #return filtered_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CoffeeLake_01\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CoffeeLake_01\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [402]:
tfidf_vectorizer = TfidfVectorizer(input='filename',max_df=1.0, max_features=10000000, encoding='utf-8',
                                 min_df=0.0, stop_words='english', decode_error='replace',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))
import glob
t = [f for f in glob.glob("txt_autores_id/*.txt", recursive=False)]
#t = !ls txt2/*.txt
%time tfidf_matrix = tfidf_vectorizer.fit_transform(t)

print(tfidf_matrix.shape)

Wall time: 28.5 s
(100, 1042179)


In [403]:
terms = tfidf_vectorizer.get_feature_names()
terms[:10]

['aaa',
 'aaa build',
 'aaa build hand',
 'aaa page',
 'aaa page essenti',
 'aaa portland',
 'aaa portland gardenfor',
 'aaa spring',
 'aaa spring symp',
 'aaai']

In [404]:
with open('terms.txt','w', encoding='utf-8') as f:
    f.write('\n'.join(terms))

In [405]:
x = tfidf_matrix.toarray()

In [406]:
df = pd.DataFrame(x,columns=terms,index=[re.sub('txt?/(\d+.?\d+v\d+)\.txt','\\1',a) for a in t])
df.head()

,aaa,aaa build,aaa build hand,aaa page,aaa page essenti,aaa portland,aaa portland gardenfor,aaa spring,aaa spring symp,aaai,...,zytnicki osac cooper,zytnicki reason,zytnicki reason superior,zytnicki unfortun,zytnicki unfortun worst,zytnicki zytnicki,zytnicki zytnicki gaspin,zzm,zzm comput,zzm comput equival
txt_autores_id\100199_104.txt,0.001404,0.0,0.0,0.001583,0.001583,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
txt_autores_id\100219_153.txt,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
txt_autores_id\100220_153.txt,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
txt_autores_id\100221_153.txt,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
txt_autores_id\100228_6.txt,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [407]:
df['comunidade'] = df.index
df['comunidade'] = df['comunidade'].str.replace('txt_autores_id','')
df['comunidade'] = df['comunidade'].str.replace('\\','')
df['comunidade'] = df['comunidade'].str.replace('.txt','')
df[['id_autor', 'comunidade']] = df['comunidade'].str.split("_", n = 1, expand = True)
df.head()

,aaa,aaa build,aaa build hand,aaa page,aaa page essenti,aaa portland,aaa portland gardenfor,aaa spring,aaa spring symp,aaai,...,zytnicki reason superior,zytnicki unfortun,zytnicki unfortun worst,zytnicki zytnicki,zytnicki zytnicki gaspin,zzm,zzm comput,zzm comput equival,comunidade,id_autor
txt_autores_id\100199_104.txt,0.001404,0.0,0.0,0.001583,0.001583,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104,100199
txt_autores_id\100219_153.txt,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,153,100219
txt_autores_id\100220_153.txt,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,153,100220
txt_autores_id\100221_153.txt,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,153,100221
txt_autores_id\100228_6.txt,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,100228


In [408]:
#autorWord_community = df.drop(['id_autor','comunidade'],axis=1)
#autorWord_community = autorWord_community.replace(to_replace=0,value=np.NaN)
#autorWord_community = autorWord_community.apply(lambda a: pd.qcut(a,3,duplicates='drop'))
#autorWord_community = autorWord_community.apply(lambda a: pd.qcut(a,3,labels=["baixo","medio","alto"],duplicates='drop'))
#autorWord_community = autorWord_community.apply(lambda a: pd.qcut(a,3,labels=["baixo","medio","alto"]))

In [ ]:
autorWord_community = df.drop(['id_autor','comunidade'],axis=1)
autorWord_community = autorWord_community.replace(to_replace=0,value=np.NaN)
for c in autorWord_community.columns:
    #print(autorWord_community[c].describe())
    q1, m, q3 = autorWord_community[c].quantile([.25, .5, .75])
    cDicreta = []
    #print(q1)
    #print(m)
    #print(q3)
    for cItem in autorWord_community[c]:
        #print(cItem)
        if cItem <= q1:
            cDicreta.append('baixo')
        elif cItem >= q3:
            cDicreta.append('alto')        
        elif cItem > q1 and cItem < q3:
            cDicreta.append('medio')
        else:
            cDicreta.append(np.nan)
    #print(cDicreta)
    autorWord_community[c] = cDicreta
#for c in autorWord_community.coll:
#    c.time_diff.quantile([0.25,0.5,0.75])

In [ ]:
autorWord_community['comunidade'] = df.comunidade
autorWord_community.shape

In [ ]:
autorWord_community.head()

In [ ]:
autorWord_community.comunidade.unique()

In [ ]:
autorWord_community.fillna('zero').to_csv('arXiv_Grau_10_90_noLimit.csv', sep=',', index=False)

In [348]:
for c in autorWord_community.columns:
    print(c)
    print(autorWord_community[c].describe())


aaai
count        18
unique        3
top       medio
freq          8
Name: aaai, dtype: object
ab
count         9
unique        3
top       medio
freq          3
Name: ab, dtype: object
ab error
count        5
unique       3
top       alto
freq         2
Name: ab error, dtype: object
abbrevi
count        39
unique        3
top       medio
freq         19
Name: abbrevi, dtype: object
abc
count        17
unique        3
top       medio
freq          7
Name: abc, dtype: object
abc abc
count        13
unique        3
top       medio
freq          5
Name: abc abc, dtype: object
abc abc abc
count         7
unique        3
top       medio
freq          3
Name: abc abc abc, dtype: object
abcd
count         8
unique        3
top       medio
freq          4
Name: abcd, dtype: object
abd
count        5
unique       3
top       alto
freq         2
Name: abd, dtype: object
abduct
count        10
unique        3
top       medio
freq          4
Name: abduct, dtype: object
abe
count         7
unique  

Name: action form, dtype: object
action given
count        14
unique        3
top       medio
freq          6
Name: action given, dtype: object
action high
count        9
unique       3
top       alto
freq         4
Name: action high, dtype: object
action lead
count        13
unique        3
top       medio
freq          5
Name: action lead, dtype: object
action let
count        12
unique        3
top       medio
freq          6
Name: action let, dtype: object
action maxim
count        13
unique        3
top       medio
freq          5
Name: action maxim, dtype: object
action model
count         9
unique        3
top       medio
freq          3
Name: action model, dtype: object
action node
count        12
unique        3
top       medio
freq          6
Name: action node, dtype: object
action observ
count        12
unique        3
top       medio
freq          6
Name: action observ, dtype: object
action optim
count        11
unique        3
top       medio
freq          5
Name: action o

count       13
unique       3
top       alto
freq         5
Name: agent choos action, dtype: object
agent consid
count         8
unique        3
top       medio
freq          4
Name: agent consid, dtype: object
agent given
count        5
unique       3
top       alto
freq         2
Name: agent given, dtype: object
agent know
count        13
unique        3
top       medio
freq          5
Name: agent know, dtype: object
agent need
count        10
unique        3
top       medio
freq          4
Name: agent need, dtype: object
agent observ
count         8
unique        3
top       medio
freq          4
Name: agent observ, dtype: object
agent onli
count        5
unique       3
top       alto
freq         2
Name: agent onli, dtype: object
agent perform
count        10
unique        3
top       medio
freq          4
Name: agent perform, dtype: object
agent problem
count        5
unique       3
top       alto
freq         2
Name: agent problem, dtype: object
agent receiv
count         6
uniqu

Name: algorithm includ, dtype: object
algorithm infer
count        17
unique        3
top       medio
freq          7
Name: algorithm infer, dtype: object
algorithm input
count        16
unique        3
top       medio
freq          8
Name: algorithm input, dtype: object
algorithm instanc
count         6
unique        3
top       medio
freq          2
Name: algorithm instanc, dtype: object
algorithm iter
count        28
unique        3
top       medio
freq         11
Name: algorithm iter, dtype: object
algorithm iter join
count         6
unique        3
top       medio
freq          2
Name: algorithm iter join, dtype: object
algorithm join
count         8
unique        3
top       medio
freq          4
Name: algorithm join, dtype: object
algorithm join tree
count         8
unique        3
top       medio
freq          3
Name: algorithm join tree, dtype: object
algorithm known
count        17
unique        3
top       medio
freq          7
Name: algorithm known, dtype: object
algorithm 

count        33
unique        3
top       medio
freq         15
Name: ampl, dtype: object
anal
count        21
unique        3
top       medio
freq          9
Name: anal, dtype: object
anal ysis
count        17
unique        3
top       medio
freq          7
Name: anal ysis, dtype: object
anali
count        14
unique        3
top       medio
freq          6
Name: anali, dtype: object
analog
count        58
unique        3
top       medio
freq         28
Name: analog, dtype: object
analogu
count        14
unique        3
top       medio
freq          6
Name: analogu, dtype: object
analys
count        23
unique        3
top       medio
freq         11
Name: analys, dtype: object
analysi
count        84
unique        3
top       medio
freq         42
Name: analysi, dtype: object
analyst
count         7
unique        3
top       medio
freq          3
Name: analyst, dtype: object
analyt
count        31
unique        3
top       medio
freq         15
Name: analyt, dtype: object
analyz
count 

...

Name: valu veri, dtype: object
valu zero
count        13
unique        3
top       medio
freq          5
Name: valu zero, dtype: object
valuabl
count        41
unique        3
top       medio
freq         18
Name: valuabl, dtype: object
valuat
count        20
unique        3
top       medio
freq         10
Name: valuat, dtype: object
van
count        25
unique        3
top       medio
freq         11
Name: van, dtype: object
van der
count        11
unique        3
top       medio
freq          5
Name: van der, dtype: object
van der gaag
count         9
unique        3
top       medio
freq          3
Name: van der gaag, dtype: object
vanish
count        12
unique        3
top       medio
freq          6
Name: vanish, dtype: object
vant
count         7
unique        3
top       medio
freq          3
Name: vant, dtype: object
var
count        39
unique        3
top       medio
freq         19
Name: var, dtype: object
var var
count        14
unique        3
top       medio
freq          6


count        13
unique        3
top       medio
freq          5
Name: variabl mean, dtype: object
variabl model
count        36
unique        3
top       medio
freq         18
Name: variabl model, dtype: object
variabl moreov
count         9
unique        3
top       medio
freq          3
Name: variabl moreov, dtype: object
variabl need
count        16
unique        3
top       medio
freq          8
Name: variabl need, dtype: object
variabl network
count        29
unique        3
top       medio
freq         13
Name: variabl network, dtype: object
variabl node
count        28
unique        3
top       medio
freq         14
Name: variabl node, dtype: object
variabl non
count        20
unique        3
top       medio
freq         10
Name: variabl non, dtype: object
variabl normal
count        17
unique        3
top       medio
freq          7
Name: variabl normal, dtype: object
variabl note
count        20
unique        3
top       medio
freq         10
Name: variabl note, dtype: object


Name: veri general, dtype: object
veri good
count        20
unique        3
top       medio
freq         10
Name: veri good, dtype: object
veri hard
count        18
unique        3
top       medio
freq          8
Name: veri hard, dtype: object
veri high
count        16
unique        3
top       medio
freq          8
Name: veri high, dtype: object
veri import
count        13
unique        3
top       medio
freq          5
Name: veri import, dtype: object
veri larg
count        38
unique        3
top       medio
freq         18
Name: veri larg, dtype: object
veri littl
count        19
unique        3
top       medio
freq          9
Name: veri littl, dtype: object
veri low
count        20
unique        3
top       medio
freq         10
Name: veri low, dtype: object
veri natur
count        12
unique        3
top       medio
freq          6
Name: veri natur, dtype: object
veri similar
count        44
unique        3
top       medio
freq         19
Name: veri similar, dtype: object
veri simp

count        19
unique        3
top       medio
freq          9
Name: whatev, dtype: object
wheel
count         7
unique        3
top       medio
freq          3
Name: wheel, dtype: object
whenev
count        56
unique        3
top       medio
freq         28
Name: whenev, dtype: object
wherea
count        72
unique        3
top       medio
freq         36
Name: wherea, dtype: object
wherebi
count         8
unique        3
top       medio
freq          4
Name: wherebi, dtype: object
whi
count        54
unique        3
top       medio
freq         26
Name: whi, dtype: object
whilst
count         8
unique        3
top       medio
freq          4
Name: whilst, dtype: object
white
count        29
unique        3
top       medio
freq         13
Name: white, dtype: object
wide
count        75
unique        3
top       medio
freq         36
Name: wide, dtype: object
wide rang
count        30
unique        3
top       medio
freq         14
Name: wide rang, dtype: object
wide use
count        3

Name: zero entri, dtype: object
zero probabl
count        25
unique        3
top       medio
freq         11
Name: zero probabl, dtype: object
zero valu
count        14
unique        3
top       medio
freq          6
Name: zero valu, dtype: object
zero weight
count         6
unique        3
top       medio
freq          2
Name: zero weight, dtype: object
zhang
count        26
unique        3
top       medio
freq         12
Name: zhang, dtype: object
zhang pool
count        11
unique        3
top       medio
freq          5
Name: zhang pool, dtype: object
zilberstein
count         9
unique        3
top       medio
freq          3
Name: zilberstein, dtype: object
zkl
count        7
unique       2
top       alto
freq         5
Name: zkl, dtype: object
zkl log
count        7
unique       2
top       alto
freq         5
Name: zkl log, dtype: object
zml
count        7
unique       2
top       alto
freq         5
Name: zml, dtype: object
zoeter
count        5
unique       3
top       alto
fre